# Configuración inicial

## Pips

In [ ]:
#!pip install numpy scipy matplotlib

## Celda 1: Importación de librerías

In [ ]:
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt

## Celda 2: Definición de matrices básicas de Pauli y proyectores

In [ ]:
# Matriz Identidad (2x2)
I = np.array([[1, 0], [0, 1]], dtype=complex)

# Matriz Pauli-X
X = np.array([[0, 1], [1, 0]], dtype=complex)

# Matriz Pauli-Y
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)

# Matriz Pauli-Z
Z = np.array([[1, 0], [0, -1]], dtype=complex)

# Proyectores |0><0| y |1><1|
P0 = np.array([[1, 0], [0, 0]], dtype=complex)
P1 = np.array([[0, 0], [0, 1]], dtype=complex)

# Parte 1: Codificación SHOR-9

## Celda 3: Función para expandir operador de un qubit a N qubits

In [ ]:
# Expansión de operador de un qubit a N qubits (posición "target" en sistema de "n" qubits)
def expand_operator(op, target, n):
    """Expande una matriz de un qubit (2x2) a un sistema de n qubits, actuando en la posición 'target' (0-index)."""
    ops = [I] * n
    ops[target] = op
    return ops[0] if n == 1 else ops[0]
    res = ops[0]
    for i in range(1, n):
        res = np.kron(res, ops[i])
    return res

## Celda 4: Función CNOT para N qubits

In [ ]:
# Construcción de matriz CNOT en espacio de N qubits (control, target en 0-index)
def cnot_n(control, target, n):
    """Devuelve la matriz unitaria de una puerta CNOT con control y target específicos en un sistema de n qubits."""
    size = 2 ** n
    out = np.zeros((size, size), dtype=complex)
    for i in range(size):
        b = list(np.binary_repr(i, n))
        if b[control] == '1':
            b[target] = '0' if b[target] == '1' else '1'
        j = int(''.join(b), 2)
        out[j, i] = 1
    return out

## Celda 5: Construcción del circuito de codificación SHOR-9

In [ ]:
# Construcción de la matriz unitaria del circuito de codificación SHOR-9
def shor9_encoding_unitary():
    """
    Devuelve la matriz unitaria del circuito de codificación del código Shor de 9 qubits.

    El circuito consta de:
    - 3 puertas Hadamard (en qubits 0, 3, 6) para introducir superposición
    - 6 puertas CNOT para la codificación de repetición:
      * Bloque A (qubits 0-2): CNOT(0,1), CNOT(0,2)
      * Bloque B (qubits 3-5): CNOT(3,4), CNOT(3,5)
      * Bloque C (qubits 6-8): CNOT(6,7), CNOT(6,8)
    """
    n_qubits = 9

    # Empezar con la identidad
    U = np.eye(2**n_qubits, dtype=complex)

    # Aplicar Hadamard en los qubits 0, 3, 6 (primero de cada bloque)
    for h_qubit in [0, 3, 6]:
        H_expanded = expand_operator_to_n_qubits(np.array([[1, 1], [1, -1]], dtype=complex) / np.sqrt(2),
                                                  h_qubit, n_qubits)
        U = H_expanded @ U

    # Aplicar CNOT dentro de cada bloque para realizar la repetición
    cnot_pairs = [
        (0, 1), (0, 2),  # Bloque A
        (3, 4), (3, 5),  # Bloque B
        (6, 7), (6, 8)   # Bloque C
    ]

    for control, target in cnot_pairs:
        CNOT = cnot_n(control, target, n_qubits)
        U = CNOT @ U

    return U


def expand_operator_to_n_qubits(op, target_qubit, n_qubits):
    """
    Expande una matriz de un qubit (2x2) a un sistema de n qubits usando producto de Kronecker.

    Parámetros:
    - op: operador de un qubit (2x2)
    - target_qubit: posición del qubit donde actúa el operador (0-index)
    - n_qubits: número total de qubits en el sistema

    Retorna:
    - Matriz de dimensión (2^n_qubits x 2^n_qubits)
    """
    ops = [I if i != target_qubit else op for i in range(n_qubits)]

    result = ops[0]
    for i in range(1, n_qubits):
        result = np.kron(result, ops[i])

    return result


## Celda 6: Verificación intermedia - Código de repetición de 3 qubits

In [ ]:
# Verificación intermedia: código de repetición 3-qubits
def repetition3_encoding_unitary():
    """
    Devuelve la matriz unitaria del circuito de codificación con código de repetición de 3 qubits.

    Este es un circuito simplificado que contiene solo:
    - 1 puerta Hadamard (en qubit 0)
    - 2 puertas CNOT: CNOT(0,1), CNOT(0,2)

    Sirve como comprobación intermedia antes de construir el Shor-9 completo.
    """
    n_qubits = 3

    # Empezar con la identidad
    U = np.eye(2**n_qubits, dtype=complex)

    # Aplicar Hadamard en el qubit 0
    H = np.array([[1, 1], [1, -1]], dtype=complex) / np.sqrt(2)
    H_expanded = expand_operator_to_n_qubits(H, 0, n_qubits)
    U = H_expanded @ U

    # Aplicar CNOT(0,1) y CNOT(0,2)
    CNOT_01 = cnot_n(0, 1, n_qubits)
    U = CNOT_01 @ U

    CNOT_02 = cnot_n(0, 2, n_qubits)
    U = CNOT_02 @ U

    return U


def verify_repetition3():
    """
    Verifica que el código de repetición de 3 qubits produce los estados esperados.

    Para el estado inicial |0>, se espera: |000>/√2 + |111>/√2
    """
    print("=" * 70)
    print("VERIFICACIÓN: Código de Repetición de 3 Qubits")
    print("=" * 70)

    U_rep3 = repetition3_encoding_unitary()

    # Estado inicial |0> en el qubit 0, el resto en |0>
    psi_0 = np.zeros(2**3, dtype=complex)
    psi_0[0] = 1.0  # |000>

    # Aplicar el circuito de codificación
    psi_encoded = U_rep3 @ psi_0

    # El resultado esperado es (|000> + |111>) / √2
    expected = np.zeros(2**3, dtype=complex)
    expected[0] = 1.0 / np.sqrt(2)  # |000>
    expected[7] = 1.0 / np.sqrt(2)  # |111>

    # Calcular fidelidad
    fidelidad = np.abs(np.vdot(expected, psi_encoded))**2

    print(f"\nEstado inicial: |0⟩ = |000⟩")
    print(f"\nEstado codificado (primeros componentes):")
    for i in range(min(8, len(psi_encoded))):
        if np.abs(psi_encoded[i]) > 1e-10:
            binary = format(i, '03b')
            print(f"  |{binary}⟩: {psi_encoded[i]:.6f}")

    print(f"\nEstado esperado (|000⟩ + |111⟩) / √2:")
    print(f"  |000⟩: {1/np.sqrt(2):.6f}")
    print(f"  |111⟩: {1/np.sqrt(2):.6f}")

    print(f"\nFidelidad: {fidelidad:.6f}")
    print(f"✓ Verificación correcta" if fidelidad > 0.99 else "✗ Error en la verificación")
    print("=" * 70)

    return U_rep3, psi_encoded


## Celda 7: Estados lógicos codificados SHOR-9

In [ ]:
# Definición de los estados lógicos |0_L> y |1_L> para el código Shor-9
def logical_states_shor9():
    """
    Devuelve los estados lógicos |0_L> y |1_L> codificados en el código Shor-9.

    Según el enunciado:
    |0_L> = (1/(2√2)) * (|000> + |111>)⊗3
    |1_L> = (1/(2√2)) * (|000> - |111>)⊗3

    donde ⊗3 significa el producto tensorial repetido 3 veces.
    """
    n_qubits = 9

    # Construir el estado base para un bloque: (|000> + |111>) / √2
    block_0 = np.zeros(2**3, dtype=complex)
    block_0[0] = 1.0 / np.sqrt(2)  # |000>
    block_0[7] = 1.0 / np.sqrt(2)  # |111>

    # Construir el estado base para un bloque: (|000> - |111>) / √2
    block_1 = np.zeros(2**3, dtype=complex)
    block_1[0] = 1.0 / np.sqrt(2)   # |000>
    block_1[7] = -1.0 / np.sqrt(2)  # -|111>

    # Construir |0_L> = block_0 ⊗ block_0 ⊗ block_0
    psi_0L = np.kron(block_0, np.kron(block_0, block_0))

    # Construir |1_L> = block_1 ⊗ block_1 ⊗ block_1
    psi_1L = np.kron(block_1, np.kron(block_1, block_1))

    return psi_0L, psi_1L


def verify_logical_states_shor9():
    """
    Verifica que los estados lógicos son ortogonales y están normalizados.
    """
    print("=" * 70)
    print("VERIFICACIÓN: Estados Lógicos SHOR-9")
    print("=" * 70)

    psi_0L, psi_1L = logical_states_shor9()

    # Verificar normalización
    norm_0L = np.linalg.norm(psi_0L)
    norm_1L = np.linalg.norm(psi_1L)

    print(f"\nNormalización:")
    print(f"  ||0_L||: {norm_0L:.6f}")
    print(f"  ||1_L||: {norm_1L:.6f}")

    # Verificar ortogonalidad
    inner_product = np.vdot(psi_0L, psi_1L)

    print(f"\nOrtogonalidad:")
    print(f"  <0_L|1_L>: {inner_product:.6f}")

    # Mostrar componentes no nulas de cada estado
    print(f"\nComponentes no nulas de |0_L>:")
    for i in range(2**9):
        if np.abs(psi_0L[i]) > 1e-10:
            binary = format(i, '09b')
            print(f"  |{binary}⟩: {psi_0L[i]:.6f}")

    print(f"\nComponentes no nulas de |1_L>:")
    for i in range(2**9):
        if np.abs(psi_1L[i]) > 1e-10:
            binary = format(i, '09b')
            print(f"  |{binary}⟩: {psi_1L[i]:.6f}")

    print(f"\n✓ Estados correctamente normalizados" if (abs(norm_0L - 1.0) < 1e-10 and
                                                       abs(norm_1L - 1.0) < 1e-10) else
          "✗ Error en normalización")
    print(f"✓ Estados ortogonales" if abs(inner_product) < 1e-10 else
          "✗ Los estados no son ortogonales")
    print("=" * 70)

    return psi_0L, psi_1L


# Ejecutar verificaciones
U_rep3, psi_rep3 = verify_repetition3()
psi_0L, psi_1L = verify_logical_states_shor9()

# Construir y mostrar la unitaria completa de SHOR-9
print("\n" + "=" * 70)
print("Construcción del circuito de codificación SHOR-9")
print("=" * 70)
U_shor9 = shor9_encoding_unitary()
print(f"Dimensión de la matriz SHOR-9: {U_shor9.shape}")
print(f"¿Es unitaria?: {np.allclose(U_shor9 @ U_shor9.conj().T, np.eye(2**9))}")
print("=" * 70)


VERIFICACIÓN: Código de Repetición de 3 Qubits

Estado inicial: |0⟩ = |000⟩

Estado codificado (primeros componentes):
  |000⟩: 0.707107+0.000000j
  |111⟩: 0.707107+0.000000j

Estado esperado (|000⟩ + |111⟩) / √2:
  |000⟩: 0.707107
  |111⟩: 0.707107

Fidelidad: 1.000000
✓ Verificación correcta
VERIFICACIÓN: Estados Lógicos SHOR-9

Normalización:
  ||0_L||: 1.000000
  ||1_L||: 1.000000

Ortogonalidad:
  <0_L|1_L>: 0.000000+0.000000j

Componentes no nulas de |0_L>:
  |000000000⟩: 0.353553+0.000000j
  |000000111⟩: 0.353553+0.000000j
  |000111000⟩: 0.353553+0.000000j
  |000111111⟩: 0.353553+0.000000j
  |111000000⟩: 0.353553+0.000000j
  |111000111⟩: 0.353553+0.000000j
  |111111000⟩: 0.353553+0.000000j
  |111111111⟩: 0.353553+0.000000j

Componentes no nulas de |1_L>:
  |000000000⟩: 0.353553+0.000000j
  |000000111⟩: -0.353553+0.000000j
  |000111000⟩: -0.353553+0.000000j
  |000111111⟩: 0.353553+0.000000j
  |111000000⟩: -0.353553+0.000000j
  |111000111⟩: 0.353553-0.000000j
  |111111000⟩: 0.3535

# Parte 2: Cálculo del Síndrome - Código SHOR-9

## Celda 8: Definición de los 8 estabilizadores

In [ ]:
# Definición de los 8 estabilizadores del código SHOR-9
def define_stabilizers():
    """
    Define los 8 estabilizadores independientes del código SHOR-9.

    Estructura según el enunciado:
    - S1-S6: Generadores ZZ por bloques (detectan errores tipo X)
      * S1 = Z0*Z1  (Bloque A)
      * S2 = Z1*Z2  (Bloque A)
      * S3 = Z3*Z4  (Bloque B)
      * S4 = Z4*Z5  (Bloque B)
      * S5 = Z6*Z7  (Bloque C)
      * S6 = Z7*Z8  (Bloque C)

    - S7-S8: Generadores X solapados (detectan errores tipo Z)
      * S7 = X0*X1*X2*X3*X4*X5  (Bloques A y B, qubits 0-5)
      * S8 = X3*X4*X5*X6*X7*X8  (Bloques B y C, qubits 3-8)
    """
    n_qubits = 9
    stabilizers = {}

    # Estabilizadores ZZ por bloques (bit-flip detection)
    # Bloque A (qubits 0-2)
    stabilizers['S1'] = pauli_string_to_matrix('ZZIIIIIII', n_qubits)  # Z0*Z1
    stabilizers['S2'] = pauli_string_to_matrix('IZZIIIIII', n_qubits)  # Z1*Z2

    # Bloque B (qubits 3-5)
    stabilizers['S3'] = pauli_string_to_matrix('IIIIZZIII', n_qubits)  # Z3*Z4
    stabilizers['S4'] = pauli_string_to_matrix('IIIIIZZII', n_qubits)  # Z4*Z5

    # Bloque C (qubits 6-8)
    stabilizers['S5'] = pauli_string_to_matrix('IIIIIIZZZ', n_qubits)  # Z6*Z7 ← CORREGIDO
    stabilizers['S6'] = pauli_string_to_matrix('IIIIIIIZZ', n_qubits)  # Z7*Z8 ← CORREGIDO

    # Estabilizadores X solapados (phase-flip detection)
    # S7: X sobre qubits 0-5 (Bloques A y B)
    stabilizers['S7'] = pauli_string_to_matrix('XXXXXXIII', n_qubits)

    # S8: X sobre qubits 3-8 (Bloques B y C)
    stabilizers['S8'] = pauli_string_to_matrix('IIIXXXXXX', n_qubits)

    return stabilizers



def pauli_string_to_matrix(pauli_string, n_qubits):
    """
    Convierte una cadena de caracteres (I, X, Y, Z) a una matriz de Pauli en un sistema de n qubits.

    Ejemplo: 'ZZIIIIIII' representa Z⊗Z⊗I⊗I⊗I⊗I⊗I⊗I⊗I

    Parámetros:
    - pauli_string: cadena con caracteres I, X, Y, Z
    - n_qubits: número de qubits en el sistema

    Retorna:
    - Matriz de dimensión (2^n_qubits x 2^n_qubits)
    """
    if len(pauli_string) != n_qubits:
        raise ValueError(f"La cadena debe tener {n_qubits} caracteres, pero tiene {len(pauli_string)}: '{pauli_string}'")

    pauli_map = {'I': I, 'X': X, 'Y': Y, 'Z': Z}

    result = pauli_map[pauli_string[0]]
    for i in range(1, n_qubits):
        result = np.kron(result, pauli_map[pauli_string[i]])

    return result


# Verificar los estabilizadores
print("=" * 70)
print("Definición de Estabilizadores del Código SHOR-9")
print("=" * 70)
stabilizers = define_stabilizers()
for name, matrix in stabilizers.items():
    # Verificar que sea hermitiano (propiedad de los estabilizadores)
    is_hermitian = np.allclose(matrix, matrix.conj().T)
    print(f"{name}: Hermitiano={is_hermitian}, Dim={matrix.shape}")
print("=" * 70)


Definición de Estabilizadores del Código SHOR-9
S1: Hermitiano=True, Dim=(512, 512)
S2: Hermitiano=True, Dim=(512, 512)
S3: Hermitiano=True, Dim=(512, 512)
S4: Hermitiano=True, Dim=(512, 512)
S5: Hermitiano=True, Dim=(512, 512)
S6: Hermitiano=True, Dim=(512, 512)
S7: Hermitiano=True, Dim=(512, 512)
S8: Hermitiano=True, Dim=(512, 512)


## Celda 9: Función para determinar conmutación/anticonmutación

In [ ]:
# Función auxiliar para determinar conmutación/anticonmutación de operadores Pauli
def pauli_commutation(P, Q):
    """
    Determina si dos operadores de Pauli conmutan o anticonmutan.

    Utiliza la tabla de conmutación de Pauli:
    - Mismo tipo (X-X, Y-Y, Z-Z): conmutan
    - Diferente tipo (X-Y, X-Z, Y-Z, etc.): anticonmutan
    - Con identidad: siempre conmutan

    Parámetros:
    - P: operador de Pauli (cadena: I, X, Y, Z)
    - Q: operador de Pauli (cadena: I, X, Y, Z)

    Retorna:
    - True si conmutan, False si anticonmutan
    """
    commutation_table = {
        ('I', 'I'): True,   ('I', 'X'): True,   ('I', 'Y'): True,   ('I', 'Z'): True,
        ('X', 'I'): True,   ('X', 'X'): True,   ('X', 'Y'): False,  ('X', 'Z'): False,
        ('Y', 'I'): True,   ('Y', 'X'): False,  ('Y', 'Y'): True,   ('Y', 'Z'): False,
        ('Z', 'I'): True,   ('Z', 'X'): False,  ('Z', 'Y'): False,  ('Z', 'Z'): True,
    }
    return commutation_table.get((P, Q), None)


def check_pauli_commutation_matrix(P_matrix, Q_matrix):
    """
    Determina si dos matrices de Pauli conmutan o anticonmutan.

    Verifica si [P, Q] = PQ - QP = 0 (conmutan) o {P, Q} = PQ + QP = 0 (anticonmutan).

    Parámetros:
    - P_matrix: matriz de Pauli
    - Q_matrix: matriz de Pauli

    Retorna:
    - 1 si conmutan
    - -1 si anticonmutan
    """
    commutator = P_matrix @ Q_matrix - Q_matrix @ P_matrix
    anticommutator = P_matrix @ Q_matrix + Q_matrix @ P_matrix

    if np.allclose(commutator, 0):
        return 1
    elif np.allclose(anticommutator, 0):
        return -1
    else:
        # Caso donde no es una relación pura de conmutación
        return None


print("\n" + "=" * 70)
print("Tabla de Conmutación de Operadores Pauli")
print("=" * 70)
paulis = ['I', 'X', 'Y', 'Z']
print("P\\Q\t", end="")
for q in paulis:
    print(f"{q}\t", end="")
print()
for p in paulis:
    print(f"{p}\t", end="")
    for q in paulis:
        result = "✓" if pauli_commutation(p, q) else "✗"
        print(f"{result}\t", end="")
    print()
print("=" * 70)



Tabla de Conmutación de Operadores Pauli
P\Q	I	X	Y	Z	
I	✓	✓	✓	✓	
X	✓	✓	✗	✗	
Y	✓	✗	✓	✗	
Z	✓	✗	✗	✓	


## Celda 10: Función para calcular el síndrome

In [ ]:
# Cálculo del síndrome para el código SHOR-9
def calculate_syndrome(state_vector, stabilizers):
    """
    Calcula el síndrome de un estado cuántico dado.

    El síndrome es un vector de 8 valores ±1 obtenidos midiendo los eigenvalores
    del estado con respecto a cada estabilizador.

    Para un estado |ψ⟩ sin error, se tiene S_i|ψ⟩ = |ψ⟩ (eigenvalor +1).
    Para un estado con error E, se tiene S_i(E|ψ⟩) = ±E|ψ⟩ según conmutación.

    Parámetros:
    - state_vector: estado cuántico (vector de dimensión 2^9)
    - stabilizers: diccionario con los 8 estabilizadores

    Retorna:
    - syndrome: vector de 8 valores ±1
    """
    syndrome = {}
    stabilizer_names = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8']

    for i, name in enumerate(stabilizer_names):
        S = stabilizers[name]

        # Aplicar el estabilizador al estado: S|ψ⟩
        result = S @ state_vector

        # Calcular el eigenvalor: λ = ⟨ψ|S|ψ⟩ / ⟨ψ|ψ⟩
        eigenvalue = np.real(np.vdot(state_vector, result) / np.vdot(state_vector, state_vector))

        # Redondear a ±1
        if eigenvalue > 0:
            syndrome[name] = 1
        else:
            syndrome[name] = -1

    return syndrome


def syndrome_to_array(syndrome):
    """
    Convierte el diccionario de síndrome a un array de 8 elementos.

    Retorna: [s1, s2, s3, s4, s5, s6, s7, s8]
    """
    stabilizer_names = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8']
    return np.array([syndrome[name] for name in stabilizer_names], dtype=int)


print("\n" + "=" * 70)
print("Función de Cálculo de Síndrome")
print("=" * 70)
print("El síndrome permite identificar el tipo y ubicación del error.")
print("Se calcula como: s_i = ⟨ψ|S_i|ψ⟩ / ⟨ψ|ψ⟩")
print("=" * 70)



Función de Cálculo de Síndrome
El síndrome permite identificar el tipo y ubicación del error.
Se calcula como: s_i = ⟨ψ|S_i|ψ⟩ / ⟨ψ|ψ⟩


## Celda 11: Función para aplicar errores

In [ ]:
# Aplicación de errores de Pauli en qubits específicos
def apply_pauli_error(state_vector, error_type, qubit, n_qubits=9):
    """
    Aplica un error de Pauli (X, Y, Z) en un qubit específico.

    Parámetros:
    - state_vector: estado cuántico inicial
    - error_type: tipo de error ('X', 'Y', 'Z')
    - qubit: posición del qubit donde se aplica el error (0-8)
    - n_qubits: número total de qubits (por defecto 9)

    Retorna:
    - state_vector_error: estado con el error aplicado
    """
    error_map = {'X': X, 'Y': Y, 'Z': Z}

    if error_type not in error_map:
        raise ValueError("Error type debe ser 'X', 'Y' o 'Z'")

    if qubit < 0 or qubit >= n_qubits:
        raise ValueError(f"Qubit debe estar entre 0 y {n_qubits-1}")

    # Expandir el operador de error a n qubits
    error_matrix = expand_operator_to_n_qubits(error_map[error_type], qubit, n_qubits)

    # Aplicar el error: E|ψ⟩
    state_with_error = error_matrix @ state_vector

    return state_with_error


print("\n" + "=" * 70)
print("Función para Aplicar Errores de Pauli")
print("=" * 70)
print("Permite simular errores X, Y, Z en cualquier qubit del sistema.")
print("=" * 70)



Función para Aplicar Errores de Pauli
Permite simular errores X, Y, Z en cualquier qubit del sistema.


## Celda 12: Pruebas del cálculo de síndrome

In [ ]:
# Pruebas exhaustivas del cálculo de síndrome
def test_syndrome_calculation():
    """
    Realiza pruebas exhaustivas de cálculo de síndrome para todos los errores
    tipo X, Y, Z en cada qubit del sistema.
    """
    print("\n" + "=" * 70)
    print("PRUEBAS: Cálculo de Síndrome para Errores Individuales")
    print("=" * 70)

    stabilizers = define_stabilizers()
    psi_0L, psi_1L = logical_states_shor9()

    error_types = ['X', 'Y', 'Z']
    results = {}

    # Probar cada tipo de error en cada qubit
    for error_type in error_types:
        print(f"\n{'─' * 70}")
        print(f"Errores tipo {error_type}:")
        print(f"{'─' * 70}")
        results[error_type] = {}

        for qubit in range(9):
            # Aplicar el error al estado lógico |0_L>
            state_with_error = apply_pauli_error(psi_0L, error_type, qubit)

            # Calcular el síndrome
            syndrome = calculate_syndrome(state_with_error, stabilizers)
            syndrome_array = syndrome_to_array(syndrome)

            results[error_type][qubit] = syndrome_array

            # Mostrar resultados
            bloque = qubit // 3  # 0, 1, o 2 para bloques A, B, C
            posicion_bloque = qubit % 3  # 0, 1, o 2 dentro del bloque
            bloques_nombres = ['A', 'B', 'C']

            print(f"Error {error_type} en qubit {qubit} (Bloque {bloques_nombres[bloque]}, posición {posicion_bloque}):")
            print(f"  Síndrome: {syndrome_array}")
            print(f"  Detalles: S1={syndrome['S1']:+d} S2={syndrome['S2']:+d} S3={syndrome['S3']:+d} S4={syndrome['S4']:+d} " +
                  f"S5={syndrome['S5']:+d} S6={syndrome['S6']:+d} S7={syndrome['S7']:+d} S8={syndrome['S8']:+d}")

    print("\n" + "=" * 70)
    print("Resumen de Síndromes por Tipo de Error")
    print("=" * 70)

    # Crear tabla resumen
    for error_type in error_types:
        print(f"\nErrores {error_type}:")
        print("Qubit | Síndrome")
        print("─" * 40)
        for qubit in range(9):
            syndrome_str = ' '.join([str(s) for s in results[error_type][qubit]])
            print(f"  {qubit}   | {syndrome_str}")

    print("=" * 70)

    return results


# Ejecutar pruebas
syndrome_test_results = test_syndrome_calculation()



PRUEBAS: Cálculo de Síndrome para Errores Individuales

──────────────────────────────────────────────────────────────────────
Errores tipo X:
──────────────────────────────────────────────────────────────────────
Error X en qubit 0 (Bloque A, posición 0):
  Síndrome: [-1  1  1 -1 -1  1  1  1]
  Detalles: S1=-1 S2=+1 S3=+1 S4=-1 S5=-1 S6=+1 S7=+1 S8=+1
Error X en qubit 1 (Bloque A, posición 1):
  Síndrome: [-1 -1  1 -1 -1  1  1  1]
  Detalles: S1=-1 S2=-1 S3=+1 S4=-1 S5=-1 S6=+1 S7=+1 S8=+1
Error X en qubit 2 (Bloque A, posición 2):
  Síndrome: [ 1 -1  1 -1 -1  1  1  1]
  Detalles: S1=+1 S2=-1 S3=+1 S4=-1 S5=-1 S6=+1 S7=+1 S8=+1
Error X en qubit 3 (Bloque B, posición 0):
  Síndrome: [ 1  1  1 -1 -1  1  1  1]
  Detalles: S1=+1 S2=+1 S3=+1 S4=-1 S5=-1 S6=+1 S7=+1 S8=+1
Error X en qubit 4 (Bloque B, posición 1):
  Síndrome: [ 1  1 -1 -1 -1  1  1  1]
  Detalles: S1=+1 S2=+1 S3=-1 S4=-1 S5=-1 S6=+1 S7=+1 S8=+1
Error X en qubit 5 (Bloque B, posición 2):
  Síndrome: [ 1  1 -1 -1 -1  1  1  1]

# Parte 3: Recuperación del Error en el Código SHOR-9

En esta sección se implementará la recuperación del error detectado en la parte 2. A partir del síndrome calculado, se identificará el tipo y la posición del error para aplicar la corrección adecuada. Al finalizar, se verificará la fidelidad del estado corregido con el estado original codificado.



## Estabilizadores y Significado del Síndrome

Los 8 estabilizadores del código SHOR-9 se dividen en dos grupos principales:

- $S_1$ a $S_6$: Detectan errores tipo $X$ (bit-flip) en los bloques $A$, $B$ y $C$.  
- $S_7$ y $S_8$: Detectan errores tipo $Z$ (phase-flip) entre bloques.

El síndrome es un vector de 8 bits $\mathbf{s} = (s_1, s_2, \ldots, s_8)$, donde cada $s_i \in \{\pm 1\}$. La interpretación de los pares de bits es la siguiente:

Para detectar errores $X$ dentro de un bloque (pares $(s_1, s_2)$, $(s_3, s_4)$, $(s_5, s_6)$):

\[
\begin{cases}
(+1, +1) &\to \text{No error en el bloque} \\
(-1, +1) &\to \text{Error en el primer qubit del bloque} \\
(+1, -1) &\to \text{Error en el segundo qubit del bloque} \\
(-1, -1) &\to \text{Error en el tercer qubit del bloque}
\end{cases}
\]

Para errores $Z$ entre bloques (par $(s_7, s_8)$):

\[
\begin{cases}
(+1, +1) &\to \text{No error} \\
(-1, +1) &\to \text{Error en primer bloque} \\
(+1, -1) &\to \text{Error en segundo bloque} \\
(-1, -1) &\to \text{Error en tercer bloque}
\end{cases}
\]


## Lógica de Interpretación del Síndrome

El síndrome $\mathbf{s}$ obtenido se procesa para determinar el error presente:

1. Interpretar los pares $(s_1, s_2)$, $(s_3, s_4)$ y $(s_5, s_6)$ para detectar errores $X$ en los bloques respectivos.
2. Interpretar $(s_7, s_8)$ para detectar errores $Z$ entre bloques.
3. Si el síndrome indica simultáneamente errores $X$ y $Z$ en la misma posición de qubit, se diagnostica un error tipo $Y$ ($Y = iXZ$).
4. Si el síndrome es $(+1, +1, +1, +1, +1, +1, +1, +1)$ no hay error.
5. Para patrones inconsistentes o múltiples errores, el código no garantiza corrección.

Esta lógica permite asignar un único tipo de error en un qubit físico particular.


## Aplicación Conceptual de la Corrección

Una vez identificado el error, se aplica la corrección inversa:

- Para error $X$ en un qubit, aplicar la compuerta $X$ en dicho qubit.  
- Para error $Z$, aplicar la compuerta $Z$.  
- Para error $Y$, aplicar la compuerta $Y$.  

Esto devuelve el estado al subespacio libre de errores, recuperando el estado lógico original, debido a la propiedad correctora del código SHOR-9.

Si hay múltiples errores, la corrección puede fallar, ya que el código solo corrige un error arbitrario.


## Decodificación y Prueba de Fidelidad

Posterior a la corrección, el estado debe ser decodificado a un solo qubit lógico. Se comparará el estado decodificado con el estado lógico original usando la fidelidad:

$$
[
F(\rho_\text{original}, \rho_\text{corregido}) = \left( \operatorname{Tr} \sqrt{ \sqrt{\rho_\text{original}} \rho_\text{corregido} \sqrt{\rho_\text{original}} } \right)^2
]
$$

Donde $\rho_\text{original}$ es el estado puro lógico codificado y $\rho_\text{corregido}$ el estado después de la corrección y decodificación.

Una fidelidad cercana a 1 indica corrección exitosa.


## Celda 1: Función para interpretar el síndrome y determinar tipo y posición de error

In [ ]:
def interpret_syndrome(s_array):
    """
    Interpreta el síndrome (array de 8 valores ±1) para identificar
    el tipo de error (X, Y, Z) y el índice del qubit exacto.

    Mapeo teórico:
    - Para cada par de estabilizadores ZZ en un bloque:
      * (-1, +1) → primer qubit del bloque
      * (+1, -1) → segundo qubit del bloque
      * (-1, -1) → tercer qubit del bloque
      * (+1, +1) → sin error en ese bloque
    """
    error_type = None
    qubit_index = None

    # Bloque A (qubits 0-2): detectado por S1, S2
    s1, s2 = s_array[0], s_array[1]
    if (s1, s2) == (-1, +1):
        error_type, qubit_index = 'X', 0
    elif (s1, s2) == (-1, -1):
        error_type, qubit_index = 'X', 1
    elif (s1, s2) == (+1, -1):
        error_type, qubit_index = 'X', 2

    # Bloque B (qubits 3-5): detectado por S3, S4
    s3, s4 = s_array[2], s_array[3]
    if error_type is None:
        if (s3, s4) == (-1, +1):
            error_type, qubit_index = 'X', 3
        elif (s3, s4) == (-1, -1):
            error_type, qubit_index = 'X', 4
        elif (s3, s4) == (+1, -1):
            error_type, qubit_index = 'X', 5

    # Bloque C (qubits 6-8): detectado por S5, S6
    s5, s6 = s_array[4], s_array[5]
    if error_type is None:
        if (s5, s6) == (-1, +1):
            error_type, qubit_index = 'X', 6
        elif (s5, s6) == (-1, -1):
            error_type, qubit_index = 'X', 7
        elif (s5, s6) == (+1, -1):
            error_type, qubit_index = 'X', 8

    # Detectar errores tipo Z usando S7 y S8
    s7, s8 = s_array[6], s_array[7]
    z_detected = False
    z_block_start = None

    if (s7, s8) == (-1, +1):
        z_detected = True
        z_block_start = 0  # Bloque A
    elif (s7, s8) == (-1, -1):
        z_detected = True
        z_block_start = 3  # Bloque B
    elif (s7, s8) == (+1, -1):
        z_detected = True
        z_block_start = 6  # Bloque C

    # Si solo hay error Z (sin error X previo)
    if error_type is None and z_detected:
        error_type = 'Z'
        qubit_index = z_block_start  # Asignar al primer qubit del bloque

    # Si hay tanto error X como Z en el mismo bloque → error Y
    if error_type == 'X' and z_detected:
        # Verificar que el qubit detectado por X está en el mismo bloque que Z
        if z_block_start <= qubit_index < z_block_start + 3:
            error_type = 'Y'

    return error_type, qubit_index



## Celda 2: Función para aplicar la corrección según el síndrome identificado

In [ ]:
def apply_correction(psi_error, error_type, qubit_index):
    """
    Aplica el operador de corrección inverso (X, Y o Z) al estado erróneo.
    """
    if error_type is None or qubit_index is None:
        return psi_error

    return apply_pauli_error(psi_error, error_type, qubit_index)



## Celda 3: Función general de recuperación (detección + corrección)


In [ ]:
def recover_state(psi_error):
    """
    Calcula el síndrome del estado, interpreta el error y aplica la corrección correspondiente.
    """
    stabilizers = define_stabilizers()
    syndrome = calculate_syndrome(psi_error, stabilizers)
    s_array = syndrome_to_array(syndrome)

    error_type, qubit_index = interpret_syndrome(s_array)
    psi_corrected = apply_correction(psi_error, error_type, qubit_index)

    print(f"Detected error: {error_type} on qubit {qubit_index}")
    print(f"Syndrome: {s_array}")

    return psi_corrected, error_type, qubit_index


## Celda 4: Decodificación del estado corregido

In [ ]:
# Celda 4 (CORREGIDA): Decodificación del estado corregido

def decode_state(psi_encoded):
    """
    Decodifica el estado corregido extrayendo la información lógica del estado codificado.

    En lugar de aplicar la inversa de la unitaria completa, se proyecta el estado
    en el subespacio lógico (primeros 2 estados) para extraer el qubit lógico.

    La decodificación extrae información de los primeros qubits de cada bloque
    (qubits 0, 3, 6) que llevan la información lógica.
    """
    # Normalizar el estado de entrada
    psi_encoded = psi_encoded / np.linalg.norm(psi_encoded)

    # Proyectar el estado codificado al subespacio lógico de 1 qubit
    # El estado lógico |0_L> tiene componentes en configuraciones |000...000>
    # El estado lógico |1_L> tiene componentes en otras configuraciones

    # Extraer proyección en base lógica
    # |0_L> corresponde a índices con patrón 0b???000??? (primeros bits de cada bloque = 0)
    # |1_L> corresponde a índices con patrón 0b???111??? (primeros bits de cada bloque = 1)

    psi_decoded = np.zeros(2, dtype=complex)

    # Componente |0_L>: suma de amplitudes donde los primeros bits de cada bloque son 0
    for i in range(2**9):
        # Extraer bits de posición 0, 3, 6 (primeros de cada bloque)
        bit0 = (i >> 0) & 1
        bit3 = (i >> 3) & 1
        bit6 = (i >> 6) & 1

        if bit0 == 0 and bit3 == 0 and bit6 == 0:
            psi_decoded[0] += psi_encoded[i]
        elif bit0 == 1 and bit3 == 1 and bit6 == 1:
            psi_decoded[1] += psi_encoded[i]

    # Normalizar
    psi_decoded = psi_decoded / np.linalg.norm(psi_decoded)

    return psi_decoded



## Celda 5: Calculo de la fidelidad

In [ ]:
def compute_fidelity(psi_original, psi_decoded):
    """
    Calcula la fidelidad F = |<ψ_original | ψ_decoded>|²
    """
    return np.abs(np.vdot(psi_original, psi_decoded))**2


## Celda 6: Ejemplo de recuperación

In [ ]:

# Estado lógico inicial |0_L>
psi_0L, psi_1L = logical_states_shor9()
psi_initial = psi_0L.copy()

# Decodificar el estado inicial para comparación posterior
psi_initial_decoded = decode_state(psi_initial)

print("="*60)
print("PRUEBAS DE CORRECCIÓN DE ERRORES EN CÓDIGO SHOR-9")
print("="*60)
print(f"Estado inicial |0_L> (codificado): {psi_initial[:4]}... (norm: {np.linalg.norm(psi_initial):.6f})")
print(f"Estado inicial |0_L> (decodificado): {psi_initial_decoded} (norm: {np.linalg.norm(psi_initial_decoded):.6f})")

# Lista de errores a probar
test_cases = [
    ('X', 1, "Error X en qubit 1 (bloque A)"),
    ('Z', 4, "Error Z en qubit 4 (bloque B)"),
    ('Y', 7, "Error Y en qubit 7 (bloque C)")
]

for i, (error_type, qubit_idx, description) in enumerate(test_cases, 1):
    print(f"\n" + "-"*50)
    print(f"EJEMPLO {i}: {description}")
    print("-"*50)

    # Aplicar error
    psi_with_error = apply_pauli_error(psi_initial, error_type, qubit_idx)
    print(f"Estado con error {error_type}{qubit_idx}: {psi_with_error[:4]}... (norm: {np.linalg.norm(psi_with_error):.6f})")

    # Verificar que el error realmente cambió el estado
    initial_error_fidelity = compute_fidelity(psi_initial, psi_with_error)
    print(f"Fidelidad inicial vs error (codificado): {initial_error_fidelity:.6f}")

    # Calcular síndrome, tipo de error y corregir
    psi_corrected, detected_type, detected_qubit = recover_state(psi_with_error)
    print(f"Estado corregido: {psi_corrected[:4]}... (norm: {np.linalg.norm(psi_corrected):.6f})")

    # Decodificar el estado corregido
    psi_decoded = decode_state(psi_corrected)
    print(f"Estado decodificado: {psi_decoded} (norm: {np.linalg.norm(psi_decoded):.6f})")

    # Calcular fidelidades
    recovery_fidelity = compute_fidelity(psi_initial, psi_corrected)
    final_fidelity = compute_fidelity(psi_initial_decoded, psi_decoded)  # ← CORREGIDO: comparar estados decodificados

    print(f"\nResultados del ejemplo {i}:")
    print(f"  Error aplicado: {error_type} en qubit {qubit_idx}")
    print(f"  Error detectado: {detected_type} en qubit {detected_qubit}")
    print(f"  Detección correcta: {'✓' if (detected_type == error_type and detected_qubit == qubit_idx) else '✗'}")
    print(f"  Fidelidad tras corrección (codificado): {recovery_fidelity:.6f}")
    print(f"  Fidelidad final (decodificado): {final_fidelity:.6f}")
    print(f"  Recuperación exitosa: {'✓' if final_fidelity > 0.99 else '✗'}")

print(f"\n" + "="*60)
print("RESUMEN DE CORRECCIÓN DE ERRORES COMPLETADO")
print("="*60)



PRUEBAS DE CORRECCIÓN DE ERRORES EN CÓDIGO SHOR-9
Estado inicial |0_L> (codificado): [0.35355339+0.j 0.        +0.j 0.        +0.j 0.        +0.j]... (norm: 1.000000)
Estado inicial |0_L> (decodificado): [0.70710678+0.j 0.70710678+0.j] (norm: 1.000000)

--------------------------------------------------
EJEMPLO 1: Error X en qubit 1 (bloque A)
--------------------------------------------------
Estado con error X1: [0.+0.j 0.+0.j 0.+0.j 0.+0.j]... (norm: 1.000000)
Fidelidad inicial vs error (codificado): 0.000000
Detected error: X on qubit 1
Syndrome: [-1 -1  1 -1 -1  1  1  1]
Estado corregido: [0.35355339+0.j 0.        +0.j 0.        +0.j 0.        +0.j]... (norm: 1.000000)
Estado decodificado: [0.70710678+0.j 0.70710678+0.j] (norm: 1.000000)

Resultados del ejemplo 1:
  Error aplicado: X en qubit 1
  Error detectado: X en qubit 1
  Detección correcta: ✓
  Fidelidad tras corrección (codificado): 1.000000
  Fidelidad final (decodificado): 1.000000
  Recuperación exitosa: ✓

------------